In [1]:
import os
import sys
import re
import csv
import json

from collections import defaultdict, Counter
from statistics import mean
import datetime, time
from tqdm.auto import tqdm

In [2]:
ru_stop = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'оно', 'когда-то', 'так',
           'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'внутри', 
           'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли',
           'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь',
           'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы',
           'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж',
           'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'почти', 'мой',
           'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'об',
           'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много',
           'разве', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя',
           'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между', 'каждый', 'пусть', 'наш', 'один', 'лишь', 'твой',
           'это', 'свой', 'весь',
          'как-то', 'кто-то', 'любой', 'просто', 'самый', 'ваш', 'столь', 'из-за', 'который', 'что-нибудь',]


ru_stop = ru_stop + list('бгдеёзйлмнпртфхцчшщьъыэю') #ивясакуож

### 1. news to zeroes
### 2. poems to zeroes

In [6]:
def dates_to_zeroes(f, year, unidict, bidict): #, month
    
    file = open(f, "r", encoding='utf-8')
    i = 0 
    corpus = []
    yearlength = int(re.search('(\d+) ', str(datetime.date(year + 1, 1, 1) - datetime.date(year, 1, 1))).group(0))
    print ("Reading file ", f)
    for line in file:
        i += 1
        year, mon, day, text = line.split('\t')
        date = datetime.date(year=int(year), month=int(mon), day=int(day))
        tokens = text.strip(' \n').split(' ')
        tokens = [token for token in tokens if token not in ru_stop]
        
        for token in tokens:
            if len(unidict[token]) < 1:
                unidict[token] = [0 for i in range(yearlength)]
            
            if date != datetime.date(int(year), 1, 1):
                position = int(re.search('(\d+) ', str(date - datetime.date(int(year), 1, 1))).group(0))
            else:
                position = 0
            unidict[token][position] = 1
                    
        for num, token in enumerate(tokens[1:]):
            bigram = ' '.join([tokens[num-1], tokens[num]])
            if len(bidict[bigram]) == 0:
                bidict[bigram] = [0 for i in range(yearlength)]
            if date != datetime.date(int(year), 1, 1):
                position = int(re.search('(\d+) ', str(date - datetime.date(int(year), 1, 1))).group(0))
            else:
                position = 0
            bidict[bigram][position] = 1
                    
        
        if i % 10000 == 0:
            sys.stderr.write("Reading sentence " + str(i) + "\n") # just a status message: str(i) turns the integer i into a string, so that we can concatenate it
      
    return unidict, bidict

In [ ]:
year = 2008
news = [f'{year}vz.tsv', f'{year}lenta.tsv']
links = [os.path.join(r'F:\c21\save', file) for file in news]

# zeroes
news_uz = defaultdict(list)
news_bz = defaultdict(list)

start = time.time()
for file in links:
    news_uz, news_bz = dates_to_zeroes(file, int(year), news_uz, news_bz)
end = time.time()


print(end - start) #prev 185

Reading file  F:\c21\save\2008vz.tsv


Reading sentence 10000
Reading sentence 20000
Reading sentence 30000
Reading sentence 40000
Reading sentence 50000
Reading sentence 60000


In [6]:
def poems_to_zeroes(f, year, unidict, bidict, poemdict, poembidict): #, month
    
    file = open(f, "r", encoding='utf-8')
    i = 0 
    corpus = []
    yearlength = int(re.search('(\d+) ', str(datetime.date(year + 1, 1, 1) - datetime.date(year, 1, 1))).group(0))
    print ("Reading file ", f)
    for line in file:
        i += 1
        year, mon, day, text = line.split('\t')
        date = datetime.date(year=int(year), month=int(mon), day=int(day))
        tokens = text.strip(' \n').split(' ')
        tokens = [token for token in tokens if token not in ru_stop]
        
        for token in tokens:
            if token in unidict:
                if len(poemdict[token]) < 1:
                    poemdict[token] = [0 for i in range(yearlength)]
                
                if date != datetime.date(int(year), 1, 1):
                    position = int(re.search('(\d+) ', str(date - datetime.date(int(year), 1, 1))).group(0))
                else:
                    position = 0
                poemdict[token][position] = 1
            else:
                pass
                    
        for num, token in enumerate(tokens[1:]):
            bigram = ' '.join([tokens[num-1], tokens[num]])
            if bigram in bidict:
                if len(poembidict[bigram]) == 0:
                    poembidict[bigram] = [0 for i in range(yearlength)]
                if date != datetime.date(int(year), 1, 1):
                    position = int(re.search('(\d+) ', str(date - datetime.date(int(year), 1, 1))).group(0))
                else:
                    position = 0
                poembidict[bigram][position] = 1
            else:
                pass
                    
        
        if i % 10000 == 0:
            sys.stderr.write("Reading sentence " + str(i) + "\n") # just a status message: str(i) turns the integer i into a string, so that we can concatenate it
      
    return poemdict, poembidict

In [ ]:
poems = [
f'mystem-{year}-01.tsv',
f'mystem-{year}-02.tsv',
f'mystem-{year}-03.tsv',
f'mystem-{year}-04.tsv',
f'mystem-{year}-05.tsv',
f'mystem-{year}-06.tsv',
f'mystem-{year}-07.tsv',
f'mystem-{year}-08.tsv',
f'mystem-{year}-09.tsv',
f'mystem-{year}-10.tsv',
f'mystem-{year}-11.tsv',
f'mystem-{year}-12.tsv'
]
p_links = [os.path.join(r'F:\c21\poems_lemmatized', file) for file in poems]

poems_uz = defaultdict(list)
poems_bz = defaultdict(list)

start = time.time()
for file in p_links:
    poems_uz, poems_bz = poems_to_zeroes(file, int(year), news_uz, news_bz, poems_uz, poems_bz)
end = time.time()
print(end - start)

### 1/2 to .5 dump dicts

In [11]:
with open(f'news-u{year}.json', 'w', encoding='utf-8') as t:
    t.write('{\n')
    iswritten = 0
    for key, value in news_uz.items():
        if sum(value)>1:
            if iswritten == 1:
                t.write(',\n')
                t.write(f'"{key}":{value}')
            else:
                t.write(f'"{key}":{value}')
                iswritten = 1
    t.write('\n}\n')

with open(f'news-b{year}.json', 'w', encoding='utf-8') as t:
    t.write('{\n')
    iswritten = 0
    for key, value in tqdm(news_bz.items()):
        if sum(value)>1:
            if iswritten == 1:
                t.write(',\n')
                t.write(f'"{key}":{value}')
            else:
                t.write(f'"{key}":{value}')
                iswritten = 1
    t.write('\n}\n')

with open(f'poems-u{year}.json', 'w', encoding='utf-8') as t:
    t.write('{\n')
    iswritten = 0
    for key, value in poems_uz.items():
        if sum(value)>1:
            if iswritten == 1:
                    t.write(',\n')
                    t.write(f'"{key}":{value}')
            else:
                t.write(f'"{key}":{value}')
                iswritten = 1
    t.write('\n}\n')

with open(f'poems-b{year}.json', 'w', encoding='utf-8') as t:
    t.write('{\n')
    iswritten = 0
    for key, value in tqdm(poems_bz.items()):
        if sum(value)>1:
            if iswritten == 1:
                t.write(',\n')
                t.write(f'"{key}":{value}')
            else:
                t.write(f'"{key}":{value}')
                iswritten = 1
    t.write('\n}\n')

  0%|          | 0/4350552 [00:00<?, ?it/s]

  0%|          | 0/937532 [00:00<?, ?it/s]

In [23]:
ddict = {1: [0,1,5], 90:[0,1,1], 45:[0,1,0]}
with open(f't.json', 'w', encoding='utf-8') as t:
    t.write('{\n')
    length = 0
    iswritten = 0
    for key, value in tqdm(ddict.items()):
        if sum(value)>1:
            if iswritten == 1:
                t.write(',\n')
                t.write(f'"{key}":{value}')
            else:
                t.write(f'"{key}":{value}')
                iswritten = 1
    t.write('\n')
    t.write('}\n')

{



  0%|          | 0/3 [00:00<?, ?it/s]

"1":[0, 1, 5],

"90":[0, 1, 1],

}



### 3. split dict

In [6]:
def split_year(yeardict, lag=10):
    splitted_dict = {}
    yearlength = len(next(iter(yeardict.values())))
    amount = yearlength//lag
    if yearlength%lag >0:
        amount += 1
    k = amount
    n = yearlength
    for token in tqdm(yeardict):
        chunks = []
        for i in range(k):
            chunks.append(yeardict[token][i*lag : (i+1)*lag])
        splitted_dict[token] = chunks
    return splitted_dict

In [5]:
split_news_u = split_year(news_uz)
del news_uz

  0%|          | 0/188918 [00:00<?, ?it/s]

In [6]:
split_poems_u = split_year(poems_uz)
del poems_uz

  0%|          | 0/71410 [00:00<?, ?it/s]

In [7]:
split_news_b = split_year(news_bz)
del news_bz

  0%|          | 0/1394558 [00:00<?, ?it/s]

In [8]:
split_poems_b = split_year(poems_bz)
del poems_bz

  0%|          | 0/364207 [00:00<?, ?it/s]

In [11]:
#del split_news_u
#del split_poems_u

### 4. find rises

In [9]:
def find_rise(occurences, ma_days = 3, difference = 2):
    mas = []
    rises = set()
    for occ_num, occ in enumerate(occurences):
        ma = []
        for i in range(ma_days):
            index = occ_num -i
            if index >= 0:
                ma.append(sum(occurences[index]))
        #print(occ_num, ma, sum(ma)/len(ma))
        ma = sum(ma)/len(ma)
        if len(mas) > 1 and sum(occurences[occ_num]) > mas[-1] + difference:  # вот тут вопрос как сделать
            rises.add(occ_num)
        mas.append(ma)
    return rises

In [9]:
with open(f'rises_u{year}-3.tsv', 'w', encoding='utf-8') as f:
    with open(f'rises_u_reverse{year}-3.tsv', 'w', encoding='utf-8') as r:
        for token in split_news_u:
            if token in split_poems_u:
                news = find_rise(split_news_u[token], 3, 3)
                poems = find_rise(split_poems_u[token], 3, 3)
                check = poems and news
                check2 = news and len(poems) == 0
                if check:
                    checked = '\t'.join([(datetime.date(int(year), 1, 1) + datetime.timedelta(it*10)).strftime("%Y-%m-%d") for it in sorted(check)])
                    strng = f'{token}\t{checked}\n'
                    f.write(strng)
                if check2:
                    checked = '\t'.join([(datetime.date(int(year), 1, 1) + datetime.timedelta(it*10)).strftime("%Y-%m-%d") for it in sorted(news)])
                    strng = f'{token}\t{checked}\n'
                    r.write(strng)

In [11]:
with open(f'rises_b{year}-3.tsv', 'w', encoding='utf-8') as f:
    with open(f'rises_b_reverse{year}-3.tsv', 'w', encoding='utf-8') as r:
        for token in split_news_b:
            if token in split_poems_b:
                news = find_rise(split_news_b[token], 3, 3)
                poems = find_rise(split_poems_b[token], 3, 3)
                check = poems and news
                check2 = news and len(poems) == 0
                if check:
                    checked = '\t'.join([(datetime.date(int(year), 1, 1) + datetime.timedelta(it*10)).strftime("%Y-%m-%d") for it in sorted(check)])
                    strng = f'{token}\t{checked}\n'
                    f.write(strng)
                if check2:
                    checked = '\t'.join([(datetime.date(int(year), 1, 1) + datetime.timedelta(it*10)).strftime("%Y-%m-%d") for it in sorted(news)])
                    strng = f'{token}\t{checked}\n'
                    r.write(strng)

#### 4.5 rises not presented in poems

In [10]:
with open(f'rises_u{year}NOPOEM.tsv', 'w', encoding='utf-8') as f:
    for token in split_news_u:
        if token not in split_poems_u:
            news = find_rise(split_news_u[token])
            if news:
                checked = '\t'.join([(datetime.date(int(year), 1, 1) + datetime.timedelta(it*10)).strftime("%Y-%m-%d") for it in sorted(news)])
                strng = f'{token}\t{checked}\n'
                f.write(strng)

In [18]:
with open(f'rises_b{year}NOPOEM.tsv', 'w', encoding='utf-8') as f:
    for token in split_news_b:
        if token not in split_poems_b:
            news = find_rise(split_news_b[token])
            if news:
                checked = '\t'.join([(datetime.date(int(year), 1, 1) + datetime.timedelta(it*10)).strftime("%Y-%m-%d") for it in sorted(news)])
                strng = f'{token}\t{checked}\n'
                f.write(strng)

### 5. (2.5.) read & write preprocessed (zeroes)

In [3]:
year = 2008

In [2]:
with open(f'poems-u{year}.json', encoding ='utf8') as json_file: 
    poems_uz = json.load(json_file) 
    
with open(f'news-u{year}.json', encoding ='utf8') as json_file: 
    news_uz = json.load(json_file) 

In [4]:
with open(f'news-b{year}.json', encoding ='utf8') as json_file: 
    news_bz = json.load(json_file) 

In [5]:
with open(f'poems-b{year}.json', encoding ='utf8') as json_file: 
    poems_bz = json.load(json_file) 